In [3]:
import cv2
import numpy as np

# ------------------------------
# Camera Init
# ------------------------------
cap = cv2.VideoCapture(0)

# state variables
mode = "raw"
contrast = 1.0
brightness = 0

help_text = (
    "[q] Quit  [0] Raw  [1] Gray  [2] HSV  [c/v] Contrast +/-  [b/n] Brightness +/-\n"
    "[g] Gaussian Blur  [f] Bilateral Filter  [e] Canny  [l] Hough Lines  [h] Histogram\n"
    "[t] Translate  [r] Rotate  [s] Scale  [p] Panorama  [k] Calibrate  [a] AR"
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply contrast & brightness globally
    frame = cv2.convertScaleAbs(frame, alpha=contrast, beta=brightness)

    output = frame.copy()

    if mode == "gray":
        output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
        output = cv2.cvtColor(output, cv2.COLOR_GRAY2BGR)

    elif mode == "hsv":
        output = cv2.cvtColor(output, cv2.COLOR_BGR2HSV)

    elif mode == "gaussian":
        output = cv2.GaussianBlur(output, (9, 9), 0)

    elif mode == "bilateral":
        output = cv2.bilateralFilter(output, 9, 75, 75)

    elif mode == "canny":
        gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        output = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    elif mode == "hough":
        gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=50, maxLineGap=10)
        output = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        if lines is not None:
            for l in lines:
                x1, y1, x2, y2 = l[0]
                cv2.line(output, (x1,y1), (x2,y2), (0,0,255), 2)

    elif mode == "translate":
        rows, cols = output.shape[:2]
        M = np.float32([[1, 0, 50], [0, 1, 30]])
        output = cv2.warpAffine(output, M, (cols, rows))

    elif mode == "rotate":
        rows, cols = output.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
        output = cv2.warpAffine(output, M, (cols, rows))

    elif mode == "scale":
        output = cv2.resize(output, None, fx=1.5, fy=1.5)

    # Overlay help text
    y0 = 20
    for i, line in enumerate(help_text.split("\\n")):
        y = y0 + i*20
        cv2.putText(output, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0,255,0), 1, cv2.LINE_AA)

    cv2.imshow("Live", output)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('0'):
        mode = "raw"
    elif key == ord('1'):
        mode = "gray"
    elif key == ord('2'):
        mode = "hsv"
    elif key == ord('g'):
        mode = "gaussian"
    elif key == ord('f'):
        mode = "bilateral"
    elif key == ord('e'):
        mode = "canny"
    elif key == ord('l'):
        mode = "hough"
    elif key == ord('t'):
        mode = "translate"
    elif key == ord('r'):
        mode = "rotate"
    elif key == ord('s'):
        mode = "scale"
    elif key == ord('h'):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist_img = np.zeros((300, 256, 3), dtype=np.uint8)
        cv2.normalize(hist, hist, 0, 300, cv2.NORM_MINMAX)
        for x in range(1, 256):
            cv2.line(hist_img, (x-1, 300-int(hist[x-1])), (x, 300-int(hist[x])), (0,255,0), 1)
        cv2.imshow("Histogram", hist_img)
    elif key == ord('c'):
        contrast += 0.1
    elif key == ord('v'):
        contrast = max(0.1, contrast - 0.1)
    elif key == ord('b'):
        brightness += 5
    elif key == ord('n'):
        brightness -= 5
    # placeholders for panorama, calibration, AR
    elif key == ord('p'):
        mode = "panorama"
    elif key == ord('k'):
        mode = "calibrate"
    elif key == ord('a'):
        mode = "ar"

cap.release()
cv2.destroyAllWindows()


C:\Users\alsto\AppData\Local\Temp\ipykernel_23860\3047033253.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.line(hist_img, (x-1, 300-int(hist[x-1])), (x, 300-int(hist[x])), (0,255,0), 1)
